In [ ]:
# STEP 0 – CLEAN & DOWNLOAD EVERYTHING TO EXTERNAL HDD (run once)
# ----------------------------------------------------------------

from pathlib import Path
import os, shutil, zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# 0-a) set this to your external drive’s mount point + folder
external_root = Path("/Volumes/Amogh's HDD/dataset")  
if not external_root.is_dir():
    raise FileNotFoundError(f"Drive path not found: {external_root}")
    
# 0-b) cd into that folder
os.chdir(external_root)
print("Working directory:", external_root)

# 0-c) remove old dataset folders (won’t touch your .zip files)
for name in ("dataset1","dataset2","roboflow_pistols","Armed Person with Rifle.v1i.yolov8","classifier_armed_unarmed"):
    p = external_root/name
    if p.exists():
        print("🗑 Removing:", p.name)
        shutil.rmtree(p)

# 0-d) authenticate Kaggle API
api = KaggleApi(); api.authenticate()

# 0-e) download & unzip issaisasank/guns-object-detection → ./dataset1
print("⇣ Downloading guns-object-detection → dataset1/")
api.dataset_download_files("issaisasank/guns-object-detection",
                           path="dataset1", unzip=True)
print("✔ dataset1 ready")

# 0-f) download & unzip snehilsanyal/weapon-detection-test → ./dataset2
print("⇣ Downloading weapon-detection-test → dataset2/")
api.dataset_download_files("snehilsanyal/weapon-detection-test",
                           path="dataset2", unzip=True)
print("✔ dataset2 ready")

# 0-g) extract any ZIPs in this folder into their own dirs
for z in external_root.glob("*.zip"):
    out = external_root / z.stem
    if out.exists(): shutil.rmtree(out)
    print(f"⇣ Extracting {z.name} → {out.name}/")
    with zipfile.ZipFile(z) as zf:
        zf.extractall(out)
    print(f"✔ {out.name} ready")

print("\n✅ All datasets are now under", external_root)

In [ ]:
# STEP 0-A – add Roboflow pistols ZIP to the external HDD dataset
# ---------------------------------------------------------------
from pathlib import Path
import zipfile, shutil, os

# Path to your external HDD dataset root (same one used in Step 0)
base_dir = Path("/Volumes/Amogh's HDD/dataset")   # edit if different
if not base_dir.is_dir():
    raise FileNotFoundError(f"Dataset root not found: {base_dir}")

# 1️⃣ find a *.zip with “pistols” in its file name in the notebook folder

zip_path = "/Users/amogharya/Documents/gun detection multiple ds/roboflow_pistols.zip"

# 2️⃣ target extraction folder inside your HDD dataset
out_folder = base_dir / "roboflow_pistols"
if out_folder.exists():
    shutil.rmtree(out_folder)
print(f"⇣ Extracting {zip_path} → {out_folder.name}/")
with zipfile.ZipFile(zip_path) as zf:
    zf.extractall(out_folder)
print("✔ roboflow_pistols ready at", out_folder)

In [ ]:
# STEP 1 – MERGE + CLEAN (host machine, current directory)
# --------------------------------------------------------
from pathlib import Path
import shutil, random, yaml, os, sys

root = Path.cwd()
print("📂 Working directory:", root)

img_exts = {".jpg", ".jpeg", ".png"}
pairs = []

def label_for(img):
    """images/…/foo.jpg  ->  labels/…/foo.txt"""
    parts = list(img.parts)
    try:
        idx = parts.index("images")
        parts[idx] = "labels"
        return Path(*parts).with_suffix(".txt")
    except ValueError:
        return None

# 1️⃣ Scan every image file under any *images* sub-tree
dataset_hits = {}
for img in root.rglob("*"):
    if img.suffix.lower() not in img_exts:         # not jpg/png
        continue
    if "images" not in img.parts:                  # not inside images/
        continue

    lbl = label_for(img)
    if lbl and lbl.exists():
        ds_name = img.parts[ img.parts.index("images") - 1 ]  # folder above images
        dataset_hits.setdefault(ds_name, 0)
        dataset_hits[ds_name] += 1
        pairs.append((ds_name, img, lbl))

print("\n🔍 Datasets discovered & pair counts:")
for k,v in dataset_hits.items():
    print(f"   {k:<25} {v} pairs")
print(f"\nTOTAL pairs found = {len(pairs)}")

# 2️⃣ Prepare YOLO skeleton (data/train, data/val)
yolo_root = root / "data"
if yolo_root.exists():
    shutil.rmtree(yolo_root)
for split in ("train", "val"):
    (yolo_root/split/"images").mkdir(parents=True, exist_ok=True)
    (yolo_root/split/"labels").mkdir(parents=True, exist_ok=True)

# 3️⃣ Shuffle & 80/20 split
random.seed(42)
random.shuffle(pairs)
cut = int(0.8 * len(pairs))
train_pairs, val_pairs = pairs[:cut], pairs[cut:]

def copy_pairs(subset, split):
    copied = 0
    for tag, img, lbl in subset:
        stem = f"{tag}_{img.stem}"
        dst_img = yolo_root/split/"images"/f"{stem}{img.suffix.lower()}"
        dst_lbl = yolo_root/split/"labels"/f"{stem}.txt"
        shutil.copy2(img, dst_img)
        shutil.copy2(lbl, dst_lbl)
        copied += 1
        if copied % 2000 == 0:
            print(f"   ⏳ copied {copied} → {split}")  # progress log
    print(f"✔ {copied} files copied into {split}")

copy_pairs(train_pairs, "train")
copy_pairs(val_pairs,   "val")

# 4️⃣ Clean & unify labels (class → 0, drop bad rows / empty files)
def clean(lbl_dir, img_dir):
    drops = 0
    for txt in lbl_dir.glob("*.txt"):
        lines = txt.read_text().splitlines()
        good = []
        for L in lines:
            p = L.split()
            if len(p) < 5:
                continue
            _, x, y, w, h = p[:5]
            good.append(f"0 {x} {y} {w} {h}")
        if good:
            txt.write_text("\n".join(good) + "\n")
        else:
            txt.unlink()
            drops += 1
            for ext in img_exts:
                (img_dir/f"{txt.stem}{ext}").unlink(missing_ok=True)
    return drops

dropped_train = clean(yolo_root/"train"/"labels", yolo_root/"train"/"images")
dropped_val   = clean(yolo_root/"val"/"labels",   yolo_root/"val"/"images")

print(f"🧹 Dropped {dropped_train} empty/bad labels from train, {dropped_val} from val")

# 5️⃣ Write data.yaml
data_yaml = root/"data.yaml"
yaml.safe_dump({
    "train": str((yolo_root/"train"/"images").resolve()),
    "val":   str((yolo_root/"val"/"images").resolve()),
    "nc":    1,
    "names": ["gun"]
}, open(data_yaml, "w"))

print("\n📝 data.yaml saved →", data_yaml)
print("✅ Merge & clean complete — you can now run the training step.")

In [ ]:
# STEP 4 – Train YOLO-v8 NANO on the merged dataset (host Mac, no aug, imgsz = 352)
# --------------------------------------------------------------------------------
from ultralytics import YOLO
import torch, time, warnings, multiprocessing as mp
from pathlib import Path

# 1️⃣  Paths
root      = Path.cwd()                   # <— you copied the whole dataset here
data_yaml = root / "data.yaml"           # written by Step 1
assert data_yaml.exists(), f"{data_yaml} not found"

runs_dir  = root / "runs"                # training outputs stay on your SSD
runs_dir.mkdir(exist_ok=True)

# 2️⃣  Device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("🖥  Training on:", device)

# 3️⃣  Model
model = YOLO("yolov8n.pt")               # nano → Pi-friendly

# 4️⃣  Base config (no on-the-fly aug, disk cache)
base = dict(
    data       = str(data_yaml),
    imgsz      = 352,          # divisible by 32, ~350
    epochs     = 20,
    device     = device,
    cache      = "disk",
    workers    = mp.cpu_count(),
    amp        = True,
    augment    = False,
    close_mosaic = 0,
    cos_lr     = True,
    val        = False,        # skip per-epoch val to keep it fast
    pretrained = True,
    patience   = 5,
    project    = str(runs_dir),
    name       = "gun_nano_352_host",
    verbose    = True,
    plots      = False,
    save_period = 1            # save weights every epoch
)

# 5️⃣  Try batches until one fits memory (16 → 8 → 4 → 2 → 1)
def train_until_fits(batches=(16, 8, 4, 2, 1)):
    for bs in batches:
        try:
            print(f"\n🚀  Launching training with batch = {bs}")
            t0 = time.time()
            model.train(batch=bs, **base)
            print(f"✅  batch={bs} finished in {(time.time()-t0)/60:.1f} min")
            return
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                warnings.warn(f"⚠️  OOM at batch {bs} – trying smaller …")
                if device == "cuda":
                    torch.cuda.empty_cache()
            else:
                raise
    raise RuntimeError("All batch sizes OOM — lower imgsz or free RAM")

train_until_fits()

print("\n🏁  Done!  Best weights →",
      runs_dir / "gun_nano_352_host" / "weights" / "best.pt")

In [ ]:
# STEP 5 – quick test on arbitrary images
# ---------------------------------------
from ultralytics import YOLO
from pathlib import Path

# 1️⃣  folders
src_dir = Path("test_imgs")           # drop images here
dst_dir = Path("test_out")
dst_dir.mkdir(exist_ok=True)

# 2️⃣  load trained weights (edit path if your run name differs)
weights = "/Users/amogharya/Documents/gun detection multiple ds/best.pt"
model   = YOLO(weights)

# 3️⃣  run inference on every image in test_imgs/
print("\nImage".ljust(30), "Result")
print("-"*45)
for img in sorted(src_dir.glob("*")):
    if img.suffix.lower() not in {".jpg", ".jpeg", ".png"}:
        continue
    r = model(img, imgsz=320, conf=0.35, save=True,
              project=dst_dir, name="")[0]
    n = len(r.boxes)
    status = f"DETECTED ({n})" if n else "NOT-DETECTED"
    print(f"{img.name.ljust(30)} {status}")

print(f"\n✔ Annotated copies saved in: {dst_dir.resolve()}")

In [ ]:
# REAL-TIME WEAPON DETECTION WITH CONTINUOUS LOGGING & CLEAN SHUTDOWN
# -----------------------------------------------------------------
import cv2
import time
import torch
import logging                               # 🔄 NEW
from ultralytics import YOLO

# 📓 Configure continuous logging  🔄 NEW
logging.basicConfig(
    filename="weapon_detection.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# 1️⃣ Select device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"🔌 Using device: {device}")

# 2️⃣ Load your best weights
model = YOLO("/Users/amogharya/Documents/gun detection multiple ds/best.pt")  # adjust path if needed

# 🔄 NEW -------------------------------------------------------------
def open_camera() -> cv2.VideoCapture | None:
    """
    Try several back-ends / indices so the script works on macOS (AVFoundation),
    Windows (MSMF) and Linux (V4L2).
    """
    tried = []
    # List of (index, backend) pairs to test
    candidates = [
        (0, cv2.CAP_AVFOUNDATION),   # macOS
        (0, cv2.CAP_MSMF),           # Windows 10+
        (0, cv2.CAP_DSHOW),          # Windows fallback
        (0, cv2.CAP_V4L2),           # Linux
        (0, None),                   # OpenCV default
        (1, None), (2, None)         # extra indices if multiple cameras
    ]
    for idx, backend in candidates:
        if backend is None:
            cap = cv2.VideoCapture(idx)
            tried.append(f"{idx}/DEFAULT")
        else:
            cap = cv2.VideoCapture(idx, backend)
            tried.append(f"{idx}/{backend}")
        if cap.isOpened():
            print(f"📸  Camera opened with {tried[-1]}")
            return cap
        cap.release()
    print(f"❌ Tried camera back-ends: {', '.join(tried)}")
    return None
# --------------------------------------------------------------------

# 3️⃣ Open webcam
cap = open_camera()                           # 🔄 NEW
if cap is None:
    raise RuntimeError("⚠️ Could not open webcam")

print("▶️  Starting real-time detection. Press q or Ctrl+C to stop.")

prev_time = time.time()
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️  Frame read failed")
            logging.warning("Frame read failed")   # 🔄 NEW
            break

        # 4️⃣ Inference
        results = model(frame, device=device, imgsz=320, conf=0.30)[0]
        n = len(results.boxes)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] Detected {n} object(s)")
        logging.info(f"Detected {n} object(s)")     # 🔄 NEW

        # 5️⃣ Draw boxes + confidences
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            logging.info(                          # 🔄 NEW
                f"Box {x1},{y1},{x2},{y2} conf={conf:.2f}"
            )

        # 6️⃣ Compute & display FPS
        now = time.time()
        fps = 1 / (now - prev_time)
        prev_time = now
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

        # 7️⃣ Show the frame
        cv2.imshow("Real-Time Gun Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("❎  Quit requested, exiting.")
            break

except KeyboardInterrupt:
    print("\n✋ Interrupted by user. Shutting down.")

finally:
    # 8️⃣ Cleanup
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Camera released and all windows closed.")

In [7]:
# ======================= Block #1: dependency + demo =======================
# ❶  Install requirements  (comment these out if already installed)
# %pip install --upgrade ultralytics==8.2.0 mediapipe opencv-python --quiet

import cv2
import torch
from ultralytics import YOLO
import mediapipe as mp
import numpy as np
from pathlib import Path

# -------- YOLOv8 initialisation ------------------------------------------------
yolo_model = YOLO(Path("best.pt"))   # make sure best.pt is in the working dir
CONF_THRES = 0.40                    # we'll tweak later

# -------- MediaPipe Pose initialisation ---------------------------------------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# -------- Video loop -----------------------------------------------------------
cap = cv2.VideoCapture(0)            # change to "sample.mp4" if needed
assert cap.isOpened(), "Cannot open video source"

while True:
    ret, frame = cap.read()
    if not ret:
        break
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # --- YOLO inference
    detections = yolo_model.predict(img_rgb, verbose=False, conf=CONF_THRES)[0]
    for box in detections.boxes:
        cls_id = int(box.cls.item())
        conf   = float(box.conf.item())
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = f"{yolo_model.names[cls_id]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # --- MediaPipe Pose inference
    res = pose.process(img_rgb)
    if res.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            frame, res.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("YOLO + MediaPipe demo", frame)
    if cv2.waitKey(1) & 0xFF == 27:   # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()
# ===========================================================================

I0000 00:00:1751930820.892451 24835907 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
W0000 00:00:1751930820.947897 24959512 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751930820.954273 24959523 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

In [ ]:
# ======================= Block #3: colours + 10-s logs ======================
import cv2, math, time, logging, numpy as np
from ultralytics import YOLO
import mediapipe as mp
from pathlib import Path

# ---------- CONFIG -----------------------------------------------------------------
YOLO_CONF   = 0.40         
ELBOW_MINDEG= 140
EXT_RATIO   = 1.10
POSE_DET_TH = 0.50
DEVICE_VID  = 0             # webcam; change to "clip.mp4" etc.

# ---------- LOGGING ----------------------------------------------------------------
logging.basicConfig(
    level    = logging.INFO,
    format   = "%(asctime)s %(levelname)s: %(message)s",
    datefmt  = "%H:%M:%S",
)

# ---------- Models -----------------------------------------------------------------
yolo = YOLO(Path("best.pt"))
mp_pose = mp.solutions.pose
pose    = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=POSE_DET_TH,
    min_tracking_confidence=0.5,
)

# ---------- Helper functions -------------------------------------------------------
def _angle(a,b,c):
    ab = np.array([a.x-b.x, a.y-b.y])
    cb = np.array([c.x-b.x, c.y-b.y])
    return math.degrees(
        math.acos(
            np.clip(np.dot(ab,cb) /
                    ((np.linalg.norm(ab)*np.linalg.norm(cb))+1e-7), -1.0, 1.0)
        )
    )

def arm_aiming(lm, side):
    if side=="L":
        shoulder, elbow, wrist, hip = lm[11], lm[13], lm[15], lm[23]
    else:
        shoulder, elbow, wrist, hip = lm[12], lm[14], lm[16], lm[24]
    if min(shoulder.visibility, elbow.visibility,
           wrist.visibility, hip.visibility) < 0.5:
        return False
    elbow_deg = _angle(shoulder, elbow, wrist)
    torso_len = math.hypot(shoulder.x-hip.x, shoulder.y-hip.y)
    arm_len   = math.hypot(shoulder.x-wrist.x, shoulder.y-wrist.y)
    return elbow_deg >= ELBOW_MINDEG and arm_len/(torso_len+1e-7) >= EXT_RATIO

def hand_box(lm, side, shape, pad=40):
    wrist = lm[15] if side=="L" else lm[16]
    h,w = shape[:2]
    cx,cy = int(wrist.x*w), int(wrist.y*h)
    return [max(cx-pad,0), max(cy-pad,0), min(cx+pad,w-1), min(cy+pad,h-1)]

# ---------- Stats & timers ---------------------------------------------------------
STATS = dict(GUN_POSE=0, GUN_ONLY=0, POSE_ONLY=0, NONE=0)
t_window_start = time.time()

def log_and_reset():
    logging.info(
        "10-s summary ➜  RED(gun+pose)=%d  ORANGE(gun)=%d  GREEN(pose)=%d  None=%d",
        STATS["GUN_POSE"], STATS["GUN_ONLY"],
        STATS["POSE_ONLY"], STATS["NONE"])
    for k in STATS: STATS[k]=0

# ---------- Colours (BGR) ----------------------------------------------------------
CLR_RED    = (  0,  0,255)
CLR_ORANGE = (  0,165,255)
CLR_GREEN  = (  0,255,  0)

# ---------- Video loop -------------------------------------------------------------
cap = cv2.VideoCapture(DEVICE_VID)
assert cap.isOpened(), "Cannot open video source"

while True:
    ok, frame = cap.read()
    if not ok: break
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ---- YOLO pass ----------------------------------------------------------------
    gun_boxes=[]
    res_det = yolo.predict(img_rgb, verbose=False, conf=YOLO_CONF)[0]
    for b in res_det.boxes:
        x1,y1,x2,y2 = map(int, b.xyxy[0])
        gun_boxes.append((x1,y1,x2,y2,float(b.conf.item())))

    # ---- Pose pass ----------------------------------------------------------------
    res_pose = pose.process(img_rgb)
    aiming   = False
    aiming_side = None
    if res_pose.pose_landmarks:
        lm = res_pose.pose_landmarks.landmark
        left = arm_aiming(lm,"L")
        right= arm_aiming(lm,"R")
        aiming = left or right
        aiming_side = "L" if left else ("R" if right else None)
        mp.solutions.drawing_utils.draw_landmarks(
            frame, res_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # ---- Decision & drawing -------------------------------------------------------
    if gun_boxes and aiming:
        event = "GUN_POSE"
        colour = CLR_RED
    elif gun_boxes:
        event = "GUN_ONLY"
        colour = CLR_ORANGE
    elif aiming:
        event = "POSE_ONLY"
        colour = CLR_GREEN
    else:
        event = "NONE"

    STATS[event] += 1    # accumulate stats

    # Draw gun boxes
    for (x1,y1,x2,y2,conf) in gun_boxes:
        cv2.rectangle(frame, (x1,y1), (x2,y2), colour, 2)
        cv2.putText(frame, f"gun {conf:.2f}", (x1,y1-6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, colour, 2)

    # Draw hand box if pose aiming
    if aiming_side:
        bx = hand_box(lm, aiming_side, frame.shape)
        cv2.rectangle(frame, (bx[0],bx[1]), (bx[2],bx[3]), colour, 2)
        cv2.putText(frame, "POSE-SHOOT", (bx[0], bx[1]-6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour, 2)

    cv2.imshow("Gun-Pose cascade", frame)
    if cv2.waitKey(1)&0xFF==27: break   # ESC

    # ---- 10-s logging -------------------------------------------------------------
    if time.time() - t_window_start >= 10:
        log_and_reset()
        t_window_start = time.time()

cap.release()
cv2.destroyAllWindows()
# =============================================================================

I0000 00:00:1752008318.900217 25541904 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
W0000 00:00:1752008318.950182 25549411 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1752008318.955101 25549412 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
16:58:48 INFO: 10-s summary ➜  RED(gun+pose)=0  ORANGE(gun)=32  GREEN(pose)=0  None=106
16:58:58 INFO: 10-s summary ➜  RED(gun+pose)=5  ORANGE(gun)=49  GREEN(pose)=3  None=62
16:59:08 INFO: 10-s summary ➜  RED(gun+pose)=0  ORANGE(gun)=75  GREEN(pose)=0  None=61
16:59:19 INFO: 10-s summary ➜  RED(gun+pose)=3  ORANGE(gun)=14  GREEN(pose)=33  None=79
16:59:29 INFO: 10-s summary ➜  RED(gun+pose)=0  ORANGE(gun)=0  GREEN(pose)=0  None=136
16:59:39 INFO: 10-s summary ➜  RED(gun+pose)=0  ORANGE(gun)=0  GREEN(pose)=0  No

KeyboardInterrupt: 

: 

In [8]:
# STEP 5 – Post-training evaluation and pretty plots (works on Ultralytics ≥ 8.3)
# ------------------------------------------------------------------------------
#
# • Runs `model.val()` on your val-split.
# • Prints mean Precision/Recall/mAP.
# • Saves two figures:
#     1)  bar chart  (per-class Precision & Recall + mAP@0.50)
#     2)  confusion-matrix heat-map  (if the matrix is exposed)
#
# Usage:
#   – Place this cell in the same notebook after STEP 4 finishes.
#   – Adjust `weights_path` / `run_name` if you changed them.

from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# ── 1  paths ──────────────────────────────────────────────────────────────
root         = Path.cwd()
weights_path = root /"best.pt"
data_yaml    = root / "data.yaml"
assert weights_path.exists(),  f"{weights_path} not found"
assert data_yaml.exists(),     f"{data_yaml} not found"

# ── 2  validate ──────────────────────────────────────────────────────────
model   = YOLO(weights_path)
metrics = model.val(data=str(data_yaml), plots=False, verbose=False)

box = metrics.box       # shorthand

mp, mr, map50, map_all = box.mp, box.mr, box.map50, box.map
print("\n──────── Validation summary ────────")
print(f"Mean Precision (mP)       : {mp:.3f}")
print(f"Mean Recall    (mR)       : {mr:.3f}")
print(f"mAP@0.50                   : {map50:.3f}")
print(f"mAP@0.50-0.95             : {map_all:.3f}")

# ── 3  per-class bar chart  (works even for 1-class models) ──────────────
cls_names = getattr(model, "names", {i:str(i) for i in range(len(box.p))})
indices   = np.arange(len(box.p))

plt.figure(figsize=(6,4))
plt.bar(indices-0.2, box.p, width=0.4, label="Precision")
plt.bar(indices+0.2, box.r, width=0.4, label="Recall")
plt.xticks(indices, [cls_names[i] for i in indices])
plt.ylim(0,1); plt.grid(axis="y", ls="--", alpha=.3)
plt.ylabel("Score"); plt.title("Per-class Precision / Recall")
for i,(p,r) in enumerate(zip(box.p, box.r)):
    plt.text(i-0.22, p+0.02, f"{p:.2f}", fontsize=8)
    plt.text(i+0.02, r+0.02, f"{r:.2f}", fontsize=8)
plt.legend()
plt.tight_layout()
plt.savefig("class_pr_bar.png", dpi=150)
plt.close()
print("📊  Saved per-class bar chart ➜ class_pr_bar.png")

# ── 4  confusion-matrix heat-map (if available) ─────────────────────────
if hasattr(metrics, "confusion_matrix"):
    cm = metrics.confusion_matrix
    fig = cm.plot(normalize=True, show_text=True, colorbar=True)
    fig.set_size_inches(4,4)
    plt.title("Normalized Confusion Matrix")
    plt.tight_layout()
    plt.savefig("conf_matrix.png", dpi=150)
    plt.close()
    print("📊  Saved confusion matrix ➜ conf_matrix.png")
else:
    print("ℹ️  confusion_matrix attribute not present – skipping CM plot")

Ultralytics 8.3.156 🚀 Python-3.12.9 torch-2.7.1 CPU (Apple M4 Pro)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 194.7±61.2 MB/s, size: 77.7 KB)


val: Scanning /Users/amogharya/Documents/gun detection multiple ds/data/val/labels.cache... 2046 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2046/2046 [00:00<?, ?it/s]

val: /Users/amogharya/Documents/gun detection multiple ds/data/val/images/train_0137710960addb4e_jpg.rf.48478cdbb6802c6463a44a14ebff6b15.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [05:13<00:00,  2.45s/it]

                   all       2046       4861      0.878      0.782      0.878      0.606
Speed: 0.1ms preprocess, 150.9ms inference, 0.0ms loss, 0.2ms postprocess per image

──────── Validation summary ────────
Mean Precision (mP)       : 0.878
Mean Recall    (mR)       : 0.782
mAP@0.50                   : 0.878
mAP@0.50-0.95             : 0.606
📊  Saved per-class bar chart ➜ class_pr_bar.png
WARNING ⚠️ ConfusionMatrix plot failure: ConfusionMatrix.plot() got an unexpected keyword argument 'show_text'


AttributeError: 'NoneType' object has no attribute 'set_size_inches'